In [1]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')

In [30]:
import sympy as sp
from liab.k_leg_liab import k_leg_liab
from liab.shapley_liab import shapley_liab
from liab.random_system import get_random_system, rerand_system, all_combs_b
from liab.failure import BooleanFormulaFailureSet
SEED = 42
NUM_VARS = 10

In [43]:
S = get_random_system(NUM_VARS, bool, seed=SEED)
T = rerand_system(S, seed=SEED+1)
print(S)
print(T)
print(S.vars_type, T.vars_type)


System(cs=[ComponentOrEquation(I=['x_0', 'x_1'], O=x_2, f=False), ComponentOrEquation(I=['x_0'], O=x_3, f=x_0), ComponentOrEquation(I=['x_0', 'x_1'], O=x_4, f=False), ComponentOrEquation(I=['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_7'], O=x_8, f=False), ComponentOrEquation(I=['x_0', 'x_2', 'x_4'], O=x_9, f=False), ComponentOrEquation(I=['x_1', 'x_2'], O=x_5, f=False), ComponentOrEquation(I=['x_2', 'x_4', 'x_5'], O=x_6, f=False), ComponentOrEquation(I=['x_4', 'x_5'], O=x_7, f=False)])
System(cs=[ComponentOrEquation(I=['x_0', 'x_1'], O=x_2, f=x_0 & x_1), ComponentOrEquation(I=['x_0'], O=x_3, f=False), ComponentOrEquation(I=['x_0', 'x_1'], O=x_4, f=x_0 & x_1), ComponentOrEquation(I=['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_7'], O=x_8, f=x_0 & x_1 & x_2 & x_3 & x_4 & x_5 & x_7), ComponentOrEquation(I=['x_0', 'x_2', 'x_4'], O=x_9, f=x_0 & x_2 & x_4), ComponentOrEquation(I=['x_1', 'x_2'], O=x_5, f=x_1 & x_2), ComponentOrEquation(I=['x_2', 'x_4', 'x_5'], O=x_6, f=x_2 & x_4 & x_5), Comp

In [45]:
M = S.induced_scm()
N = T.induced_scm()
print(M)
print(N)
print(M.vars_type, N.vars_type)

SCM(U={'x_0', 'x_1'}, V={'x_9', 'x_5', 'x_2', 'x_7', 'x_3', 'x_8', 'x_4', 'x_6'}, cs={'x_2': ComponentOrEquation(I=['x_0', 'x_1'], O=x_2, f=False), 'x_3': ComponentOrEquation(I=['x_0'], O=x_3, f=x_0), 'x_4': ComponentOrEquation(I=['x_0', 'x_1'], O=x_4, f=False), 'x_8': ComponentOrEquation(I=['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_7'], O=x_8, f=False), 'x_9': ComponentOrEquation(I=['x_0', 'x_2', 'x_4'], O=x_9, f=False), 'x_5': ComponentOrEquation(I=['x_1', 'x_2'], O=x_5, f=False), 'x_6': ComponentOrEquation(I=['x_2', 'x_4', 'x_5'], O=x_6, f=False), 'x_7': ComponentOrEquation(I=['x_4', 'x_5'], O=x_7, f=False)}, state_order=['x_3', 'x_2', 'x_4', 'x_5', 'x_9', 'x_6', 'x_7', 'x_8'])
SCM(U={'x_0', 'x_1'}, V={'x_9', 'x_5', 'x_2', 'x_7', 'x_3', 'x_8', 'x_4', 'x_6'}, cs={'x_2': ComponentOrEquation(I=['x_0', 'x_1'], O=x_2, f=x_0 & x_1), 'x_3': ComponentOrEquation(I=['x_0'], O=x_3, f=False), 'x_4': ComponentOrEquation(I=['x_0', 'x_1'], O=x_4, f=x_0 & x_1), 'x_8': ComponentOrEquation(I=['x_0

In [74]:
# F = BooleanFormulaFailureSet.get_random([str(v) for v in M.V], seed=SEED+2)
from liab.random_system import get_rand_prop

diffs = []
for i in range(1000):
    S = get_random_system(NUM_VARS, bool, seed=SEED)
    T = rerand_system(S, seed=SEED+1)
    M = S.induced_scm()
    N = T.induced_scm()
    vars = list(M.U) + list(M.V)
    for var in vars:
        locals()[var] = sp.symbols(var)
    F = BooleanFormulaFailureSet(eval(get_rand_prop(vars, num_syms=1)))

    for u_vals in all_combs_b(len(M.U)):
        u = dict(zip([str(u) for u in M.U], u_vals))
        state_m, _ = M.get_state(u)
        state_n, _ = N.get_state(u)
        if F.contains(state_m) or not F.contains(state_n):
            # print('Specification already failed or implementation not failed or both!')
            continue
        print(k)
        for k in range(1, 3):
            k_leg = k_leg_liab(T, S, u, F, k=k)
            shapley = shapley_liab(T, S, u, F, k=k)
            for var in k_leg:
                if var in shapley:
                    diffs.append(k_leg[var] - shapley[var])
                else:
                    print(f'Warning: {var} not found in Shapley liability')
        print()
    
#############
## Current problem: generating a context that M[u] \notin F but N[u] \in F
#############

2
{'x_2': 1.0, 'x_3': 0.0, 'x_4': 0.0, 'x_8': 0.0, 'x_9': 0.0, 'x_5': 0.0, 'x_6': 0.0, 'x_7': 0.0}
{'x_2': 1.0, 'x_3': 0.0, 'x_4': 0.0, 'x_8': 0.0, 'x_9': 0.0, 'x_5': 0.0, 'x_6': 0.0, 'x_7': 0.0}
{'x_2': 1.0, 'x_3': 0.0, 'x_4': 0.0, 'x_8': 0.0, 'x_9': 0.0, 'x_5': 0.0, 'x_6': 0.0, 'x_7': 0.0}
{'x_2': 1.0, 'x_3': 0.0, 'x_4': 0.0, 'x_8': 0.0, 'x_9': 0.0, 'x_5': 0.0, 'x_6': 0.0, 'x_7': 0.0}

2
{'x_2': 1.0, 'x_3': 0.0, 'x_4': 0.0, 'x_8': 0.0, 'x_9': 0.0, 'x_5': 0.0, 'x_6': 0.0, 'x_7': 0.0}


KeyboardInterrupt: 